Натренувати класифікатори на датасеті

https://archive.ics.uci.edu/ml/datasets/credit+approval

Мають бути 3 варіанти класифікаторів (стекінг, бустінг, беггінг)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
database = pd.read_csv("crx.data", sep=',', 
    names=['A1', 'A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16'])

In [3]:
database

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [4]:
print(database.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      690 non-null    object 
 1   A2      690 non-null    object 
 2   A3      690 non-null    float64
 3   A4      690 non-null    object 
 4   A5      690 non-null    object 
 5   A6      690 non-null    object 
 6   A7      690 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     690 non-null    object 
 14  A15     690 non-null    int64  
 15  A16     690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB
None


весь датасет заполнен, хоть в описании и сказано, что в 5% есть утерянные значения. в любом случае процент утерь не значителен. так как данные зашифрованы физический смысл этих дааных понять не получилось.

In [5]:
database.groupby(['A16']).count()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
A16,,,,,,,,,,,,,,,
+,307,307,307,307,307,307,307,307,307,307,307,307,307,307,307
-,383,383,383,383,383,383,383,383,383,383,383,383,383,383,383


по таргету есть небольшой дисбалланс, но не на столько, чтобы его нужно было устранять

Для сравнения разных методов будем пользоваться стандартным датасетом без использовния скалера и с один LabelEncoder , так как мы не знаем физический смысл данных

In [6]:
database['A1'] = LabelEncoder().fit_transform(database['A1'])
database['A2'] = LabelEncoder().fit_transform(database['A2'])
database['A4'] = LabelEncoder().fit_transform(database['A4'])
database['A5'] = LabelEncoder().fit_transform(database['A5'])
database['A6'] = LabelEncoder().fit_transform(database['A6'])
database['A7'] = LabelEncoder().fit_transform(database['A7'])
database['A9'] = LabelEncoder().fit_transform(database['A9'])
database['A10'] = LabelEncoder().fit_transform(database['A10'])
database['A12'] = LabelEncoder().fit_transform(database['A12'])
database['A13'] = LabelEncoder().fit_transform(database['A13'])
database['A14'] = LabelEncoder().fit_transform(database['A14'])
database['A16'] = LabelEncoder().fit_transform(database['A16'])

In [7]:
y = np.array(database['A16'])
X = database.drop(columns= ['A16', 'A2'])

In [8]:
X

,A1,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,2,0.000,2,1,13,8,1.25,1,1,1,0,0,68,0
1,1,4.460,2,1,11,4,3.04,1,1,6,0,0,11,560
2,1,0.500,2,1,11,4,1.50,1,0,0,0,0,96,824
3,2,1.540,2,1,13,8,3.75,1,1,5,1,0,31,3
4,2,5.625,2,1,13,8,1.71,1,0,0,0,2,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,2,10.085,3,3,5,4,1.25,0,0,0,0,0,90,0
686,1,0.750,2,1,2,8,2.00,0,1,2,1,0,67,394
687,1,13.500,3,3,6,3,2.00,0,1,1,1,0,67,1
688,2,0.205,2,1,1,8,0.04,0,0,0,0,0,96,750


In [9]:
#sns.pairplot(database)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

<center>Stacking</center>

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

In [12]:
estimators = [('rf', RandomForestClassifier(n_estimators=100, random_state=42)),('lr',  LinearRegression())]
clf = StackingClassifier( estimators=estimators, final_estimator=LogisticRegression())

In [13]:
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        61
           1       0.88      0.88      0.88        77

    accuracy                           0.87       138
   macro avg       0.87      0.87      0.87       138
weighted avg       0.87      0.87      0.87       138



In [14]:
estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),('lr',  LinearRegression())]
clf = StackingClassifier( estimators=estimators, final_estimator=LogisticRegression())

In [15]:
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.84      0.84        61
           1       0.87      0.88      0.88        77

    accuracy                           0.86       138
   macro avg       0.86      0.86      0.86       138
weighted avg       0.86      0.86      0.86       138



In [16]:
estimators = [('rf', RandomForestClassifier()),('lr',  LinearRegression())]
clf = StackingClassifier( estimators=estimators, final_estimator=LogisticRegression())

In [17]:
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        61
           1       0.88      0.88      0.88        77

    accuracy                           0.87       138
   macro avg       0.87      0.87      0.87       138
weighted avg       0.87      0.87      0.87       138



<center>Boosting</center>

In [18]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.88      0.85      0.87        61
           1       0.89      0.91      0.90        77

    accuracy                           0.88       138
   macro avg       0.88      0.88      0.88       138
weighted avg       0.88      0.88      0.88       138



In [19]:
clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=3, random_state=0).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82        61
           1       0.85      0.88      0.87        77

    accuracy                           0.85       138
   macro avg       0.85      0.84      0.84       138
weighted avg       0.85      0.85      0.85       138



In [20]:
clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=3).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82        61
           1       0.85      0.88      0.87        77

    accuracy                           0.85       138
   macro avg       0.85      0.84      0.84       138
weighted avg       0.85      0.85      0.85       138



In [21]:
clf = GradientBoostingClassifier().fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.84      0.87        61
           1       0.88      0.94      0.91        77

    accuracy                           0.89       138
   macro avg       0.89      0.89      0.89       138
weighted avg       0.89      0.89      0.89       138



<center>Bagging</center>

In [22]:
clf = BaggingClassifier(estimator=KNeighborsClassifier(),n_estimators=100, random_state=0).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.71      0.56      0.62        61
           1       0.70      0.82      0.75        77

    accuracy                           0.70       138
   macro avg       0.70      0.69      0.69       138
weighted avg       0.70      0.70      0.70       138



In [23]:
clf = BaggingClassifier(estimator=KNeighborsClassifier(),n_estimators=10).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.73      0.61      0.66        61
           1       0.72      0.82      0.77        77

    accuracy                           0.72       138
   macro avg       0.72      0.71      0.71       138
weighted avg       0.72      0.72      0.72       138



In [24]:
clf = BaggingClassifier(estimator=KNeighborsClassifier(),n_estimators=5, random_state=0).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.60      0.57      0.59        61
           1       0.68      0.70      0.69        77

    accuracy                           0.64       138
   macro avg       0.64      0.64      0.64       138
weighted avg       0.64      0.64      0.64       138



In [25]:
clf = BaggingClassifier(estimator=KNeighborsClassifier()).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.68      0.59      0.63        61
           1       0.71      0.78      0.74        77

    accuracy                           0.70       138
   macro avg       0.69      0.68      0.69       138
weighted avg       0.69      0.70      0.69       138



по результатам тестов с разными параметрами бэггинг дает самый худший результат(это не значит что метод плох, возможно при выборе другого эстиматора результат будет лучше, Ниже попробуем). Стэкинг с разными гиперпарамметрами выдает неплохие результаты, но при стандартных результаты заметно ухудшаются(что еще раз доказывает что подбор хороших гиперпарамметров модели играет очень важную роль). Бустинг выдает стабильно хорошие результаты, попробуем улучшить результаты по максимуму, подобрав парамметры в бустинге.

In [26]:
clf = BaggingClassifier(estimator=RandomForestClassifier()).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85        61
           1       0.86      0.91      0.89        77

    accuracy                           0.87       138
   macro avg       0.87      0.86      0.87       138
weighted avg       0.87      0.87      0.87       138



видно что метод бэггинга не плох, просто KNeighborsClassifier был не лучшим выбором для данной задачи

<center>подбор гиперпарамметров</center>

In [27]:
clf = GradientBoostingClassifier()

In [28]:
for param in clf.get_params().keys():
    print(param)

ccp_alpha
criterion
init
learning_rate
loss
max_depth
max_features
max_leaf_nodes
min_impurity_decrease
min_samples_leaf
min_samples_split
min_weight_fraction_leaf
n_estimators
n_iter_no_change
random_state
subsample
tol
validation_fraction
verbose
warm_start


In [29]:
param_grid = {'loss': ['log_loss', 'exponential'],'n_estimators': [10, 30, 100, 300], 
              'criterion': ['friedman_mse', 'squared_error'],
               'max_features':['sqrt', 'log2']}

In [30]:
reg = GridSearchCV(clf, param_grid)

In [31]:
reg.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'criterion': ['friedman_mse', 'squared_error'],
                         'loss': ['log_loss', 'exponential'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 30, 100, 300]})

In [32]:
print(classification_report(y_test, reg.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81        61
           1       0.84      0.87      0.85        77

    accuracy                           0.83       138
   macro avg       0.83      0.83      0.83       138
weighted avg       0.83      0.83      0.83       138



разница есть, но не большая от стандартных парамметров, попробуем использовать скалер

In [33]:
reg = make_pipeline(StandardScaler(),GradientBoostingClassifier())

In [34]:
reg.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [35]:
print(classification_report(y_test, reg.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.84      0.87        61
           1       0.88      0.94      0.91        77

    accuracy                           0.89       138
   macro avg       0.89      0.89      0.89       138
weighted avg       0.89      0.89      0.89       138



искользование скалера не дало никаких заметных результатов.

по завершению хочу сказать что все три варианта показали себя неплохо, для улучшения результата можно было бы узнать физический смысл закодированных данных (тогда бы можно было подобрать более подходящие варианты энкодера или разкодировать вручную) и обратить большее внимания подбору эстиматоров, так как по примерэ бэггинга это играет очень важную роль